In [24]:
pip -q install crewai  langchain openai langchain_groq langchain_community crewai_tools langchain_google_genai gradio

In [56]:
#  Set Groq API key (from Colab secrets)
import os
from google.colab import userdata
os.environ["GROQ_API_KEY"] = userdata.get("groq_key")

gemini_api_key=userdata.get("gemini_api2")

serper_api_key = userdata.get('serper_api')
os.environ["SERPER_API_KEY"] = serper_api_key

In [30]:

from crewai import LLM
# llm = LLM(model="gemini/gemini-1.5-flash", api_key=gemini_api_key)
llm = LLM(model="gemini/gemini-2.5-flash", api_key=gemini_api_key)

In [5]:
from crewai_tools import SerperDevTool ,ScrapeWebsiteTool # Google-style search tool

#Define Search Tool
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

/usr/local/lib/python3.11/dist-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


In [6]:
from crewai import Agent, Task, Crew

In [17]:


# === AGENTS ===
search_agent = Agent(
    role="Research Paper Searcher",
    goal="Find recent and relevant academic papers on a topic",
    backstory="You are skilled at finding papers from scholarly sources using search engines.",
    tools=[search_tool],
    llm=llm,
    verbose=True
)

scraper_agent = Agent(
    role="Paper Extractor",
    goal="Fetch the abstract or content from the paper URLs",
    backstory="You're a web scraper that focuses on extracting research content from paper links.",
    tools=[scrape_tool],
    llm=llm,
    verbose=True
)

summarizer_agent = Agent(
    role="Paper Summarizer",
    goal="Summarize the key points from each paper and provide a synthesized summary",
    backstory="You're an NLP expert trained to understand research papers and explain them clearly.",
    llm=llm,
    verbose=True
)

compiler_agent = Agent(
    role="Report Writer",
    goal="Compile a structured report from summaries with titles and URLs",
    backstory="You're a scientific writer focused on creating research briefs with clear structure.",
    llm=llm,
    verbose=True
)

# === TASKS ===
task1 = Task(
    description="Search and return 5 recent academic papers on '{research_topic}'. Return title and URL for each.",
    expected_output="List of 5 papers with title and link.",
    agent=search_agent
)

task2 = Task(
    description="Scrape abstracts or content from each paper URL provided by the search agent.",
    expected_output="List of full abstracts or main content from the 5 paper URLs.",
    agent=scraper_agent
)

task3 = Task(
    description=(
        "Using the content extracted from each paper, perform the following:\n\n"
        "1. For each paper:\n"
        "   - Include the full title\n"
        "   - Include the source URL\n"
        "   - Provide a detailed summary (100 words), including key methods, results, and conclusions\n\n"
        "2. After summarizing all individual papers:\n"
        "   - Write an **overall synthesized summary** that is at least 400-500 words\n"
        "    -write  overall summary of the '{research_topic}' using the information from the extracted papers "
        "   - Capture patterns, contradictions, trends, and overall contributions of the papers as a group\n"
        "   - Reflect critically on the findings (e.g. limitations, gaps, future directions)\n\n"
        "Return a dictionary with two keys:\n"
        "- `overall_summary`: a detailed paragraph combining the papers' insights\n"
        "- `paper_summaries`: a list of dicts with `title`, `url`, and `summary` for each paper"
    ),
    expected_output=(
        "A Python dictionary with:\n"
        "- overall_summary: a single detailed synthesis paragraph (400-500 words)\n"
        "- paper_summaries: list of summaries, each with title, url, summary"
    ),
    agent=summarizer_agent
)

task4 = Task(
    description=(
        "Use the `overall_summary` and `paper_summaries` from the previous task to compile a Markdown report.\n\n"
        "The report should include:\n"
        "- # Research Report: {research_topic}\n"
        "- ## 🧠 Overall Summary\n"
        "  {Insert overall_summary here}\n\n"
        "- ## 📄 Source Papers\n"
        "  For each paper in `paper_summaries`, format as:\n"
        "  ### [Title](URL)\n"
        "  **Summary:** Summary text here\n\n"
        "Generate the full markdown report based on this structure using the actual data from inputs, not template placeholders."
    ),
    expected_output="Complete markdown-formatted research report.",
    agent=compiler_agent
)

# === CREW ===
crew = Crew(
    agents=[search_agent, scraper_agent, summarizer_agent, compiler_agent],
    tasks=[task1, task2, task3, task4],
    verbose=True
)

# === RUN ===
research_topic = input("🔍 Enter your research topic: ")
result = crew.kickoff(inputs={"research_topic": research_topic})







🔍 Enter your research topic: gen ai in medical field


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: cb189bd5-e103-4dbc-baa3-5886140ab869                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Research Paper Searcher                                                                                 │
│                                                                                                                 │
│  Task: Search and return 5 recent academic papers on 'gen ai in medical field'. Return title and URL for each.  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

/usr/local/lib/python3.11/dist-packages/ipywidgets/widgets/widget_output.py:111: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  if ip and hasattr(ip, 'kernel') and hasattr(ip.kernel, '_parent_header'):

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Report Writer                                                                                           │
│                                                                                                                 │
│  Task: Use the `overall_summary` and `paper_summaries` from the previous task to compile a Markdown report.     │
│                                                                                                                 │
│  The report should include:                                                                                     │
│  - # Research Report: gen ai in medical field                                                                   │
│  - ## 🧠 Overall Summary                                                                                        │
│    {Insert overall_summary here}                                                                                │
│                                                                                                                 │
│  - ## 📄 Source Papers                                                                                          │
│    For each paper in `paper_summaries`, format as:                                                              │
│    ### [Title](URL)                                                                                             │
│    **Summary:** Summary text here                                                                               │
│                                                                                                                 │
│  Generate the full markdown report based on this structure using the actual data from inputs, not template      │
│  placeholders.                                                                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Report Writer                                                                                           │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # Research Report: gen ai in medical field                                                                     │
│  ## 🧠 Overall Summary                                                                                          │
│  The reviewed papers collectively highlight the profound and transformative potential of generative artificial  │
│  intelligence (AI) across diverse facets of medicine, signaling a paradigm shift in healthcare delivery,        │
│  research, and education. A clear overarching trend is the evolution from unimodal AI systems, such as large    │
│  language models (LLMs) primarily focused on text, towards more sophisticated multimodal AI models capable of   │
│  integrating and interpreting heterogeneous data types, including imaging, text, structured clinical data,      │
│  physiological waveforms, and multi-omics information. This progression, as detailed in several reviews,        │
│  underpins AI's expanding utility in complex medical scenarios where comprehensive data integration is          │
│  crucial.                                                                                                       │
│                                                                                                                 │
│  Across various specialties, including anatomic pathology and critical care nephrology, generative AI promises  │
│  significant enhancements. Applications span improving diagnostic accuracy through advanced image analysis and  │
│  clinical reasoning, automating routine tasks to boost workflow efficiency, and generating synthetic data for   │
│  research and education, such as synthetic histology images. LLMs, specifically, are showing promise in areas   │
│  like patient education, literature review, academic writing, and even highly specialized tasks like            │
│  troubleshooting CRRT alarms. For time-varying biomedical data, generative AI offers powerful new approaches    │
│  to trajectory modeling, overcoming limitations of traditional statistical methods in capturing complex         │
│  distributions and long-term dependencies in patient histories. The ability of these models to synthesize new   │
│  content, analyze vast datasets, and provide decision support positions them as invaluable tools for            │
│  clinicians, researchers, patients, and trainees alike.                                                         │
│                                                                                                                 │
│  Despite this immense potential, the papers consistently underscore a formidable array of challenges that must  │
│  be rigorously addressed for successful and responsible integration into clinical practice. Foremost among      │
│  these are concerns regarding trustworthiness and reliability, encompassing the need for rigorous validation    │
│  in real-world settings, ensuring model transparency and interpretability, and mitigating algorithmic bias to   │
│  uphold equitable outcomes. Data-related hurdles are also prominent, including maintaining strict patient       │
│  privacy and data security, and developing robust methods for integrating diverse and often fragmented data     │
│  types. Ethical implications, such as the necessity for continuous human oversight, are repeatedly emphasized,  │
│  asserting that AI should augment, not replace, human ex

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: af270cb1-62b8-41fc-9bd5-fd841f4c0070                                                                     │
│  Agent: Report Writer                                                                                           │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: cb189bd5-e103-4dbc-baa3-5886140ab869                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: # Research Report: gen ai in medical field                                                       │
│  ## 🧠 Overall Summary                                                                                          │
│  The reviewed papers collectively highlight the profound and transformative potential of generative artificial  │
│  intelligence (AI) across diverse facets of medicine, signaling a paradigm shift in healthcare delivery,        │
│  research, and education. A clear overarching trend is the evolution from unimodal AI systems, such as large    │
│  language models (LLMs) primarily focused on text, towards more sophisticated multimodal AI models capable of   │
│  integrating and interpreting heterogeneous data types, including imaging, text, structured clinical data,      │
│  physiological waveforms, and multi-omics information. This progression, as detailed in several reviews,        │
│  underpins AI's expanding utility in complex medical scenarios where comprehensive data integration is          │
│  crucial.                                                                                                       │
│                                                                                                                 │
│  Across various specialties, including anatomic pathology and critical care nephrology, generative AI promises  │
│  significant enhancements. Applications span improving diagnostic accuracy through advanced image analysis and  │
│  clinical reasoning, automating routine tasks to boost workflow efficiency, and generating synthetic data for   │
│  research and education, such as synthetic histology images. LLMs, specifically, are showing promise in areas   │
│  like patient education, literature review, academic writing, and even highly specialized tasks like            │
│  troubleshooting CRRT alarms. For time-varying biomedical data, generative AI offers powerful new approaches    │
│  to trajectory modeling, overcoming limitations of traditional statistical methods in capturing complex         │
│  distributions and long-term dependencies in patient histories. The ability of these models to synthesize new   │
│  content, analyze vast datasets, and provide decision support positions them as invaluable tools for            │
│  clinicians, researchers, patients, and trainees alike.                                                         │
│                                                                                                                 │
│  Despite this immense potential, the papers consistently underscore a formidable array of challenges that must  │
│  be rigorously addressed for successful and responsible integration into clinical practice. Foremost among      │
│  these are concerns regarding trustworthiness and reliability, encompassing the need for rigorous validation    │
│  in real-world settings, ensuring model transparency and interpretability, and mitigating algorithmic bias to   │
│  uphold equitable outcomes. Data-related hurdles are also prominent, including maintaining strict patient       │
│  privacy and data security, and developing robust methods for integrating diverse and often fragmented data     │
│  types. Ethical implications, such as the necessity for

In [18]:
from IPython.display import Markdown, display

# Clean raw content if it's wrapped in code block
clean_markdown = result.raw.strip("`").replace("markdown", "", 1).strip()

display(Markdown(clean_markdown))

# Research Report: gen ai in medical field
## 🧠 Overall Summary
The reviewed papers collectively highlight the profound and transformative potential of generative artificial intelligence (AI) across diverse facets of medicine, signaling a paradigm shift in healthcare delivery, research, and education. A clear overarching trend is the evolution from unimodal AI systems, such as large language models (LLMs) primarily focused on text, towards more sophisticated multimodal AI models capable of integrating and interpreting heterogeneous data types, including imaging, text, structured clinical data, physiological waveforms, and multi-omics information. This progression, as detailed in several reviews, underpins AI's expanding utility in complex medical scenarios where comprehensive data integration is crucial.

Across various specialties, including anatomic pathology and critical care nephrology, generative AI promises significant enhancements. Applications span improving diagnostic accuracy through advanced image analysis and clinical reasoning, automating routine tasks to boost workflow efficiency, and generating synthetic data for research and education, such as synthetic histology images. LLMs, specifically, are showing promise in areas like patient education, literature review, academic writing, and even highly specialized tasks like troubleshooting CRRT alarms. For time-varying biomedical data, generative AI offers powerful new approaches to trajectory modeling, overcoming limitations of traditional statistical methods in capturing complex distributions and long-term dependencies in patient histories. The ability of these models to synthesize new content, analyze vast datasets, and provide decision support positions them as invaluable tools for clinicians, researchers, patients, and trainees alike.

Despite this immense potential, the papers consistently underscore a formidable array of challenges that must be rigorously addressed for successful and responsible integration into clinical practice. Foremost among these are concerns regarding trustworthiness and reliability, encompassing the need for rigorous validation in real-world settings, ensuring model transparency and interpretability, and mitigating algorithmic bias to uphold equitable outcomes. Data-related hurdles are also prominent, including maintaining strict patient privacy and data security, and developing robust methods for integrating diverse and often fragmented data types. Ethical implications, such as the necessity for continuous human oversight, are repeatedly emphasized, asserting that AI should augment, not replace, human expertise. Furthermore, practical challenges such as seamless integration into existing electronic health records and clinical workflows, scalability of solutions, and the continuous refinement of these rapidly evolving technologies remain critical areas for development.

The collective findings point to several crucial future directions. Continued interdisciplinary collaboration between AI developers, clinicians, and ethicists is paramount to ensure that AI solutions are clinically relevant, ethically sound, and user-friendly. There is a pressing need for standardized, robust evaluation frameworks to validate AI systems in diverse clinical environments, moving beyond laboratory settings. Research must also focus on improving the interpretability of complex AI models, bridging the knowledge gap for non-developers, and developing methods to generate unbiased and representative synthetic data. Ultimately, the papers convey a cautious optimism: generative AI holds the promise to revolutionize medicine, but realizing its full, beneficial impact hinges on navigating the intricate landscape of technical, ethical, and practical challenges with unwavering commitment to patient safety and well-being.

## 📄 Source Papers
### Generative Artificial Intelligence in Anatomic Pathology (https://pubmed.ncbi.nlm.nih.gov/39836377/)
**Summary:** This paper explores the transformative potential of generative AI in anatomic pathology, aiming to enhance diagnostic accuracy, workflow efficiency, and research. It reviews unimodal and multimodal applications, showing how AI can improve image analysis, enable virtual staining, and generate synthetic data, boosting diagnostic precision. Furthermore, AI can automate routine tasks, aiding quality control and reflex testing, while also facilitating education through AI-generated content and synthetic histology images. The authors conclude that generative AI can revolutionize the field, but its successful clinical integration demands rigorous validation, ethical consideration, interdisciplinary collaboration, and adherence to patient care standards to ensure trustworthy and unbiased applications.
### Generative AI in Medicine (https://arxiv.org/html/2412.10337v2)
**Summary:** This paper provides a comprehensive overview of the expanding use cases for generative AI in medicine, targeting clinicians, patients, researchers, and trainees. It highlights the potential for AI to support diverse applications, from clinical decision-making to trial organization. However, the authors emphasize that realizing this potential hinges on overcoming significant challenges. Key hurdles include maintaining patient privacy and data security, enhancing model transparency and interpretability, ensuring equitable access and outcomes, and implementing rigorous evaluation methodologies. The paper concludes by outlining these challenges as critical open research directions that must be addressed for generative AI's responsible and effective integration into healthcare.
### Generative AI Models in Time-Varying Biomedical Data (https://pubmed.ncbi.nlm.nih.gov/40063929/)
**Summary:** This scoping review addresses the challenge of trajectory modeling in healthcare, where traditional methods struggle with complex, multimodal, time-varying biomedical data. It introduces generative AI concepts for practitioners with limited computer science background, reviewing 155 articles applying these models to time-series health data, including structured data, physiological waveforms, and medical imaging. The paper outlines how generative AI can capture intricate distributions and long-term dependencies, offering a systematic framework for clinicians to select suitable AI methods. It concludes by bridging the gap between computational methods and clinical application, identifying existing shortcomings and highlighting promising future directions for disease modeling.
### Generative AI in Critical Care Nephrology (https://pubmed.ncbi.nlm.nih.gov/39217985/)
**Summary:** This review examines the applications and future prospects of generative AI, particularly large language models (LLMs), in critical care nephrology. It demonstrates LLMs' capabilities in enhancing diagnostic accuracy, clinical reasoning, and troubleshooting for continuous renal replacement therapy (CRRT) alarms. The paper also discusses generative AI's potential in patient education, literature review, and academic writing. While highlighting opportunities for improving patient care and optimizing clinical processes, it stresses critical challenges such as ethical implications, privacy concerns, potential bias, and the imperative for human oversight. Successful integration, it concludes, requires careful consideration and continuous refinement for responsible clinical implementation.
### From large language models to multimodal AI: A scoping... (https://arxiv.org/html/2502.09242v1)
**Summary:** This scoping review traces the evolution of generative AI in medicine from unimodal large language models (LLMs) to advanced multimodal AI systems. It systematically analyzes 144 papers, revealing a significant trend towards integrating diverse data modalities like imaging, text, and structured data. The review highlights generative AI's potential in diagnostic support, medical report generation, drug discovery, and conversational AI. Despite these advancements, the paper identifies critical challenges, including integrating heterogeneous data types, improving model interpretability, addressing ethical concerns, and validating AI systems in real-world clinical settings. It concludes by summarizing the state-of-the-art, identifying gaps, and guiding the development of trustworthy and impactful multimodal AI solutions.

In [57]:
from langchain.chat_models import ChatOpenAI
from langchain_groq import ChatGroq


def generate_report(topic: str) -> str:


  # === AGENTS ===
  search_agent = Agent(
      role="Research Paper Searcher",
      goal="Find recent and relevant academic papers on a topic",
      backstory="You are skilled at finding papers from scholarly sources using search engines.",
      tools=[search_tool],
      llm=llm,
      verbose=False
  )

  scraper_agent = Agent(
      role="Paper Extractor",
      goal="Fetch the abstract or content from the paper URLs",
      backstory="You're a web scraper that focuses on extracting research content from paper links.",
      tools=[scrape_tool],
      llm=llm,
      verbose=False
  )

  summarizer_agent = Agent(
      role="Paper Summarizer",
      goal="Summarize the key points from each paper and provide a synthesized summary",
      backstory="You're an NLP expert trained to understand research papers and explain them clearly.",
      llm=llm,
      verbose=False
  )

  compiler_agent = Agent(
      role="Report Writer",
      goal="Compile a structured report from summaries with titles and URLs",
      backstory="You're a scientific writer focused on creating research briefs with clear structure.",
      llm=llm,
      verbose=False
  )

  # === TASKS ===
  task1 = Task(
      description="Search and return 5 recent academic papers on '{research_topic}'. Return title and URL for each.",
      expected_output="List of 5 papers with title and link.",
      agent=search_agent
  )

  task2 = Task(
      description="Scrape abstracts or content from each paper URL provided by the search agent.",
      expected_output="List of full abstracts or main content from the 5 paper URLs.",
      agent=scraper_agent
  )

  task3 = Task(
      description=(
          "Using the content extracted from each paper, perform the following:\n\n"
          "1. For each paper:\n"
          "   - Include the full title\n"
          "   - Include the source URL\n"
          "   - Provide a detailed summary (100 words), including key methods, results, and conclusions\n\n"
          "2. After summarizing all individual papers:\n"
          "   - Write an **overall synthesized summary** that is at least 400-500 words\n"
          "    -write  overall summary of the '{research_topic}' using the information from the extracted papers "
          "   - Capture patterns, contradictions, trends, and overall contributions of the papers as a group\n"
          "   - Reflect critically on the findings (e.g. limitations, gaps, future directions)\n\n"
          "Return a dictionary with two keys:\n"
          "- `overall_summary`: a detailed paragraph combining the papers' insights\n"
          "- `paper_summaries`: a list of dicts with `title`, `url`, and `summary` for each paper"
      ),
      expected_output=(
          "A Python dictionary with:\n"
          "- overall_summary: a single detailed synthesis paragraph (400-500 words)\n"
          "- paper_summaries: list of summaries, each with title, url, summary"
      ),
      agent=summarizer_agent
  )

  task4 = Task(
      description=(
          "Use the `overall_summary` and `paper_summaries` from the previous task to compile a Markdown report.\n\n"
          "The report should include:\n"
          "- # Research Report: {research_topic}\n"
          "- ## 🧠 Overall Summary\n"
          "  {Insert overall_summary here}\n\n"
          "- ## 📄 Source Papers\n"
          "  For each paper in `paper_summaries`, format as:\n"
          "  ### [Title](URL)\n"
          "  **Summary:** Summary text here\n\n"
          "Generate the full markdown report based on this structure using the actual data from inputs, not template placeholders."
      ),
      expected_output="Complete markdown-formatted research report.",
      agent=compiler_agent
  )


  # === CREW ===
  crew = Crew(
      agents=[search_agent, scraper_agent, summarizer_agent, compiler_agent],
      tasks=[task1, task2, task3, task4],
      verbose=False
  )


  result = crew.kickoff(inputs={"research_topic": topic})
  return result.raw.strip("`").replace("markdown", "", 1).strip()


In [43]:


import gradio as gr

# Wrap the existing report function (update name if needed)
def run_agent_ui(topic):
    return generate_report(topic)

# Gradio UI layout
with gr.Blocks() as demo:
    gr.Markdown("# 🧠 AI Research Assistant")
    gr.Markdown("This tool generates a research report using AI agents. Enter a topic below:")

    with gr.Row():
        topic_input = gr.Textbox(
            label="Research Topic",
            placeholder="e.g., Applications of CRISPR in agriculture",
            lines=1
        )

    with gr.Row():
        generate_button = gr.Button("Generate Report")

    with gr.Row():
        output_box = gr.Textbox(
            label="Generated Report",
            lines=25,
            interactive=False
        )

    generate_button.click(
        fn=run_agent_ui,
        inputs=topic_input,
        outputs=output_box
    )

# Launch the UI
demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3e1af25e5aacd5a5d8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
